# Livneh Wrfcf Root Mean Sq Error 2017-02-05 to 2017-02-17 Over California Oregon Washington Region

In [1]:
# imports
import numpy
import Nio
import Ngl
from IPython import display
import netCDF4 as nc
from datetime import date
from datetime import timedelta
import xarray as xr
import xesmf as xe
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns
import geopandas as gpd
import earthpy as et
import regionmask

In [2]:
# read in daily difference datasets
dif_05 = xr.open_dataarray('./dif_05-Copy1.nc')
dif_06 = xr.open_dataarray('./dif_06-Copy1.nc')
dif_07 = xr.open_dataarray('./dif_07-Copy1.nc')
dif_08 = xr.open_dataarray('./dif_08-Copy1.nc')
dif_09 = xr.open_dataarray('./dif_09-Copy1.nc')
dif_10 = xr.open_dataarray('./dif_10-Copy1.nc')
dif_11 = xr.open_dataarray('./dif_11-Copy1.nc')
dif_12 = xr.open_dataarray('./dif_12-Copy1.nc')
dif_13 = xr.open_dataarray('./dif_13-Copy1.nc')
dif_14 = xr.open_dataarray('./dif_14-Copy1.nc')
dif_15 = xr.open_dataarray('./dif_15-Copy1.nc')

dif_arr = [dif_05, dif_06, dif_07, dif_08, dif_09, dif_10, dif_11, dif_12, dif_13, dif_14, dif_15]

In [3]:
# rmse per point
def rmse():
    sq_arr = []
    for i in dif_arr:
        sq_dif = numpy.square(i)
        sq_arr.append(sq_dif)
    sum_arr = sq_arr[0]
    for j in sq_arr[1:]:
        sum_arr += j
    mean_sq = sum_arr / 11
    rmse = numpy.sqrt(mean_sq)
    return rmse

rm = rmse()
rm

<xarray.DataArray (lat: 444, lon: 922)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 25.16 25.22 25.28 25.34 ... 52.66 52.72 52.78 52.84
  * lon      (lon) float64 -124.6 -124.5 -124.5 -124.4 ... -67.16 -67.09 -67.03

In [4]:
# for reference
numpy.unique(rm), max(numpy.unique(rm)), min(numpy.unique(rm))

(array([0.44375986, 0.44536042, 0.51676636, ...,        nan,        nan,
               nan]),
 9.723485077954885,
 0.44375986372565357)

In [5]:
# convert array to masked array to overlook NaNs
import numpy.ma as ma
m_out = ma.masked_invalid(rm)
m_out

masked_array(
  data=[[--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        ...,
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --],
        [--, --, --, ..., --, --, --]],
  mask=[[ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        ...,
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True],
        [ True,  True,  True, ...,  True,  True,  True]],
  fill_value=1e+20)

In [6]:
# create dataarray structure
lat_coord = dif_05['lat'].data.flatten()
lon_coord = dif_05['lon'].data.flatten()
d_arr = xr.DataArray(m_out, coords = [lat_coord, lon_coord], dims = ['lat', 'lon'])
d_arr # array of rmse values per point, in case there is a need for it eventually, not end goal for this at least

<xarray.DataArray (lat: 444, lon: 922)>
array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 25.16 25.22 25.28 25.34 ... 52.66 52.72 52.78 52.84
  * lon      (lon) float64 -124.6 -124.5 -124.5 -124.4 ... -67.16 -67.09 -67.03

In [7]:
# masking california

# reading in livneh for universal mask
fn = '/cw3e/mead/projects/cwp103/scratch/ldehaan/Livneh/ptt_2017.nc'
liv = xr.open_dataset(fn)
liv_tm = liv['Tmax'][36:47]

# Download natural earth data which contains state boundaries to generate AOI
states_path = "ne_50m_admin_1_states_provinces_lakes-Copy1.shp"
states_gdf = gpd.read_file(states_path)

# You will use the bounds to determine the slice values for this data
# Select any state in the CONUS that you wish here! California is the default
cali_aoi = states_gdf[states_gdf.name == "California"]
or_aoi = states_gdf[states_gdf.name == "Oregon"]
wa_aoi = states_gdf[states_gdf.name == "Washington"]

# Create a 3d mask - this contains the true / false values identifying pixels
# inside vs outside of the mask region
cali_mask = regionmask.mask_3D_geopandas(cali_aoi,liv.lon,liv.lat)
or_mask = regionmask.mask_3D_geopandas(or_aoi, liv.lon, liv.lat)
wa_mask = regionmask.mask_3D_geopandas(wa_aoi, liv.lon, liv.lat)

In [18]:
# putting together all the masked regions differences
full = []
for i in dif_arr:
    temp = i.where(cali_mask)
    temp2 = i.where(or_mask)
    temp3 = i.where(wa_mask)
    full.append(temp)
    full.append(temp2)
    full.append(temp3)
full = numpy.array(full).flatten()
full = full[numpy.logical_not(numpy.isnan(full))]
numpy.unique(full)

array([-13.28100776, -13.2805042 , -13.09253904, ...,  16.68401432,
        16.84541312,  17.06981284])

In [19]:
# numerical data, average rmse over entire region
def overall_rmse():
    sq_arr = []
    for i in full:
        sq_dif = numpy.square(i)
        sq_arr.append(sq_dif)
    div = sum(sq_arr) / len(sq_arr)
    return numpy.sqrt(div) / 11
overall_rmse()

0.29880250034246325